---
layout: post
title: Sprint 5 Blog
description:  Here's where I will present "Sprint 5 Blog"
courses: {csp: {week: 12}}
type: issues 
comments: true
---


## Group  
Our program's goal is to create a multiplayer drawing and guessing game, inspired by Scribble.io. Players take turns drawing, while others try to guess the word or phrase being illustrated. The game aims to offer an interactive and fun experience, promoting collaboration, creativity, and quick thinking.

## Individual  
My individual contribution focuses on developing the statistics page. This feature will track key player data, such as guesses made, accuracy, and overall performance throughout the game. The goal is to provide players with valuable insights into their progress and gameplay style, adding a layer of depth and motivation to the experience. By displaying real-time statistics, the feature enhances the competitive aspect of the game, encouraging players to improve and engage more deeply with the game.


# Stats API 

## 1. List Requests, Use of Lists, Dictionaries, and Database

### List Requests
The Stats API supports the following requests:
- **GET** `/api/stats`: Retrieve the stat entries.
- **POST** `/api/stats`: Add a new stat entry.
- **PUT** `/api/stats`: Update an existing stat entry.
- **DELETE** `/api/stats/<profile_name>/<Guess>`: Delete a specific stat entry.

### Data Handling with Lists and Dictionaries

The API structures data using Python dictionaries to store individual statistics and lists to handle multiple entries. When retrieving data from the database, it is converted into a JSON-friendly format using lists of dictionaries.

#### Example: Fetching all statistics and converting them into a list of dictionaries
Here, we retrieve all statistics from the database and transform them into a list of dictionaries to be returned as a JSON response.

In [ ]:

# Fetching all stats and converting to a list of dictionaries
all_stats = Stats.query.all()
stats_list = [{
    "username": stat.user_name,
    "correct_guesses": stat.correct_guesses,
    "wrong_guesses": stat.wrong_guesses
} for stat in all_stats]



## 2. Formatting Response Data (JSON) from API into DOM
We use Flask’s jsonify function to return properly formatted JSON responses. The frontend then processes this data and updates the DOM to display relevant statistics.

#### Example: Fetching statistics data via an API endpoint
This function retrieves the first available statistics record and returns it in JSON format. If no records exist, it returns default values.

In [ ]:
@stats_api.route('/api/statistics', methods=['GET'])
def get_statistics():
    try:
        stats = Stats.query.first()
        if not stats:
            return jsonify({
                "correct_guesses": 0,
                "wrong_guesses": 0,
                "total_rounds": 0
            }), 200
        return jsonify(stats.read()), 200
    except Exception as e:
        return jsonify({"error": str(e)}), 500



## 3. Database Queries
We utilize SQLAlchemy ORM to interact with the database, allowing us to retrieve and manipulate statistics efficiently.

#### Example: Querying statistics from the database
The first query retrieves all statistics, while the second retrieves the first matching record for a given username.


In [ ]:
# Fetching all statistics entries from the database
all_stats = Stats.query.all()

# Filtering by username
stats = Stats.query.filter_by(user_name=username).first()



## 4. CRUD Methods in Class
We define CRUD (Create, Read, Update, Delete) methods in the Stats class to manage database records.

- Create: Adds a new entry to the database.
- Read: Converts a database entry to a dictionary.
- Update: Updates entry fields with new data.
- Delete: Removes an entry from the database.

#### Example:


In [ ]:
class Stats(db.Model):
    # Create
    def create(self):
        db.session.add(self)
        db.session.commit()
        
    # Read
    def read(self):
        return {
            "username": self.user_name,
            "correct_guesses": self.correct_guesses,
            "wrong_guesses": self.wrong_guesses,
            "total_rounds": self.total_rounds
        }
        
    # Update
    def update(self, data):
        for key, value in data.items():
            setattr(self, key, value)
        db.session.commit()
        
    # Delete
    def delete(self):
        db.session.delete(self)
        db.session.commit()


## 5. Algorithmic Code Request
The API allows updating statistics entries by checking if an entry exists before modifying its values.
#### Example:
This function processes incoming JSON data to either create a new statistics entry or update an existing one.

In [ ]:
@stats_api.route('/api/statistics', methods=['POST'])
def update_statistics():
    try:
        data = request.get_json()
        
        if not data or 'username' not in data:
            return jsonify({"error": "Username required"}), 400

        stats = Stats.query.filter_by(user_name=data['username']).first()
        if not stats:
            stats = Stats(
                user_name=data['username'],
                correct_guesses=int(data.get('correct', 0)),
                wrong_guesses=int(data.get('wrong', 0)),
                total_rounds=1
            )
            db.session.add(stats)
        else:
            stats.correct_guesses += int(data.get('correct', 0))
            stats.wrong_guesses += int(data.get('wrong', 0))
            stats.total_rounds += 1

        db.session.commit()
        
        all_stats = Stats.query.all()
        stats_list = [{
            "username": stat.user_name,
            "correct_guesses": stat.correct_guesses,
            "wrong_guesses": stat.wrong_guesses
        } for stat in all_stats]
        
        return jsonify(stats_list), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 6. API Class
We use Flask’s Blueprint to define the API routes. The stats_api blueprint handles the GET, POST, PUT, and DELETE methods, allowing us to organize the API endpoints and their implementations.

#### Example:


In [ ]:
stats_api = Blueprint('stats_api', __name__)

@stats_api.route('/api/statistics', methods=['GET'])
def get_statistics():
    # Implementation for GET request
    pass

@stats_api.route('/api/statistics', methods=['POST'])
def update_statistics():
    # Implementation for POST request
    pass

@stats_api.route('/api/statistics/<username>', methods=['DELETE'])
def delete_statistics(username):
    # Implementation for DELETE request
    pass


## 7. Method with Sequencing, Selection, and Iteration
The update_stats_entry method contains sequencing (steps to process the request), selection (conditional checks), and iteration (looping through data if needed). This method updates an existing entry or creates a new one based on the provided data.
#### Example:


In [ ]:
@stats_api.route('/api/statistics', methods=['PUT'])
def update_statistics():
    try:
        data = request.get_json()
        if not data or 'username' not in data:
            return jsonify({"error": "Username required"}), 400

        stats = Stats.query.filter_by(user_name=data['username']).first()
        if not stats:
            stats = Stats(
                user_name=data['username'],
                correct_guesses=int(data.get('correct', 0)),
                wrong_guesses=int(data.get('wrong', 0)),
                total_rounds=1
            )
            db.session.add(stats)
        else:
            stats.correct_guesses += int(data.get('correct', 0))
            stats.wrong_guesses += int(data.get('wrong', 0))
            stats.total_rounds += 1

        db.session.commit()

        all_stats = Stats.query.all()
        stats_list = [{
            "username": stat.user_name,
            "correct_guesses": stat.correct_guesses,
            "wrong_guesses": stat.wrong_guesses
        } for stat in all_stats]

        return jsonify(stats_list), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 8. Parameters and Return Type
The update_stats_entry method takes JSON data as input and returns a JSON response. The input data includes profile_name, game_name, and score. The response is formatted using jsonify to ensure it is returned as a JSON object.
#### Example:


In [ ]:
@stats_api.route('/api/statistics', methods=['POST'])
def update_statistics():
    try:
        data = request.get_json()
        if not data or 'username' not in data:
            return jsonify({"error": "Username required"}), 400

        stats = Stats.query.filter_by(user_name=data['username']).first()
        if not stats:
            stats = Stats(
                user_name=data['username'],
                correct_guesses=int(data.get('correct', 0)),
                wrong_guesses=int(data.get('wrong', 0)),
                total_rounds=1
            )
            db.session.add(stats)
        else:
            stats.correct_guesses += int(data.get('correct', 0))
            stats.wrong_guesses += int(data.get('wrong', 0))
            stats.total_rounds += 1

        db.session.commit()
        
        all_stats = Stats.query.all()
        stats_list = [{
            "username": stat.user_name,
            "correct_guesses": stat.correct_guesses,
            "wrong_guesses": stat.wrong_guesses
        } for stat in all_stats]

        return jsonify(stats_list), 200

    except Exception as e:
        db.session.rollback()
        return jsonify({"error": str(e)}), 500


## 9. Call to Algorithm Request
On the frontend, we use the Fetch API to communicate with the backend. To submit a score, we send a PUT request containing the profile name and score. The response is then processed by checking its status—if successful, we update the statistics; if an error occurs, an error message is displayed in the DOM.
#### Example:


In [ ]:
async function submitScore() {
    const username = document.getElementById('username').value.trim();
    const correct = parseInt(document.getElementById('correct').value);
    const wrong = parseInt(document.getElementById('wrong').value);

    if (!username) {
        showMessage('Please fill in all fields', true);
        return;
    }

    try {
        const response = await fetch(API_URL, {
            method: 'PUT',  
            headers: {
                'Content-Type': 'application/json',
            },
            body: JSON.stringify({
                username: username,
                correct: correct,
                wrong: wrong
            })
        });

        const data = await response.json();

        if (response.ok) {
            showMessage('Score updated successfully!');
            await fetchStats();
        } else {
            throw new Error(data.error || 'Failed to submit score');
        }
    } catch (error) {
        console.error('Error:', error);
        showMessage(error.message, true);
    }
}
